In [1]:
import numpy as np
import pandas as pd 
import datetime

In [2]:
df_2019 = pd.read_csv('C:\\Users\\yanis\\Downloads\\Foot test\\2019data.csv')

In [3]:
fifadata= pd.read_csv('C:\\Users\\yanis\\Downloads\\Foot test\\players_20.csv')

In [4]:
df_2019

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,F1,09/08/2019,19:45,Monaco,Lyon,0,3,A,0,2,...,2.01,0.25,2.04,1.86,2.01,1.90,2.06,2.01,2.01,1.87
1,F1,10/08/2019,16:30,Marseille,Reims,0,2,A,0,0,...,1.75,-0.75,2.02,1.88,2.05,1.87,2.06,1.97,1.98,1.89
2,F1,10/08/2019,19:00,Angers,Bordeaux,3,1,H,3,1,...,1.49,-0.25,1.75,2.05,1.79,2.15,1.80,2.17,1.77,2.11
3,F1,10/08/2019,19:00,Brest,Toulouse,1,1,D,1,0,...,1.77,-0.50,2.10,1.80,2.10,1.83,2.12,1.83,2.08,1.79
4,F1,10/08/2019,19:00,Dijon,St Etienne,1,2,A,1,2,...,1.79,0.25,2.07,1.72,2.14,1.79,2.16,1.82,2.09,1.79
5,F1,10/08/2019,19:00,Montpellier,Rennes,0,1,A,0,1,...,1.63,-0.25,1.94,1.96,1.96,1.96,1.97,2.02,1.93,1.94
6,F1,10/08/2019,19:00,Nice,Amiens,2,1,H,1,0,...,1.48,-0.25,1.98,1.92,1.99,1.93,2.00,1.95,1.96,1.91
7,F1,11/08/2019,14:00,Lille,Nantes,2,1,H,1,0,...,1.58,-0.25,1.98,1.92,2.04,1.88,2.04,2.06,1.96,1.91
8,F1,11/08/2019,16:00,Strasbourg,Metz,1,1,D,1,0,...,1.46,-0.25,1.94,1.96,1.97,1.95,1.97,2.04,1.91,1.97
9,F1,11/08/2019,20:00,Paris SG,Nimes,3,0,H,1,0,...,4.13,-3.00,2.02,1.77,2.08,1.82,2.30,1.95,2.09,1.79


In [5]:
fifadata

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,6/24/1987,170,72,Argentina,FC Barcelona,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,2/5/1985,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,2/5/1992,175,68,Brazil,Paris SG,...,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1/7/1993,188,87,Slovenia,Atlético Madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1/7/1991,175,74,Belgium,Real Madrid,...,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3
5,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,28,6/28/1991,181,70,Belgium,Manchester City,...,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3
6,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,27,4/30/1992,187,85,Germany,FC Barcelona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,203376,https://sofifa.com/player/203376/virgil-van-di...,V. van Dijk,Virgil van Dijk,27,7/8/1991,193,92,Netherlands,Liverpool,...,79+3,83+3,83+3,83+3,79+3,81+3,87+3,87+3,87+3,81+3
8,177003,https://sofifa.com/player/177003/luka-modric/2...,L. Modrić,Luka Modrić,33,9/9/1985,172,66,Croatia,Real Madrid,...,81+3,81+3,81+3,81+3,81+3,79+3,72+3,72+3,72+3,79+3
9,209331,https://sofifa.com/player/209331/mohamed-salah...,M. Salah,Mohamed Salah Ghaly,27,6/15/1992,175,71,Egypt,Liverpool,...,70+3,67+3,67+3,67+3,70+3,66+3,57+3,57+3,57+3,66+3


In [6]:
def get_winning_team(row):
    team_1 = row['HomeTeam']
    team_2 = row['AwayTeam']
    if row['FTR'] == "H":
        return team_1
    elif row['FTR'] == "D" :
        return "Draw"
    else :
        return team_2

In [7]:
def get_losing_team(row):
    team_1 = row['HomeTeam']
    team_2 = row['AwayTeam']
    if row['FTR'] == "A":
        return team_1
    elif row['FTR'] == "D" :
        return "Draw"
    else :
        return team_2
df_2019['WinningTeam'] = df_2019.apply(get_winning_team, axis=1)
df_2019['LosingTeam'] = df_2019.apply(get_losing_team, axis=1)

In [8]:
F1Teams=['Lyon','Nantes','Reims','Bordeaux','Paris SG','Lille','Marseille','Nimes','St Etienne','Montpellier','Monaco','Angers','Nice','Toulouse','Amiens','Rennes','Brest','Dijon','Strasbourg','Metz']

In [9]:
df19 = pd.DataFrame(F1Teams,columns =['Teams']) 

In [10]:
# Played Games
for i in df19.index:
    team = df19.loc[i,"Teams"]
    home_game = df_2019.groupby("HomeTeam").count().loc[team,"Div"]
    away_game = df_2019.groupby("AwayTeam").count().loc[team,"Div"]
    df19.loc[i,"Games"] = home_game + away_game

In [11]:
# Number of wins
for i in df19.index:
    team = df19.loc[i,"Teams"]
    wins = df_2019.groupby("WinningTeam").count().loc[team,"Div"]
    df19.loc[i,"Wins"] =wins
df19['WinningRatio']= df19['Wins']/df19['Games']

In [12]:
# Number of losses 
for i in df19.index:
    team = df19.loc[i,"Teams"]
    losses = df_2019.groupby("LosingTeam").count().loc[team,"Div"]
    df19.loc[i,"Losses"] = losses
df19['LosingRatio']= df19['Losses']/df19['Games']

In [13]:
# Number of Draws
df19['Draws'] = abs((df19['Wins'] + df19['Losses']) - df19['Games'])
df19['DrawRatio'] = df19['Draws']/df19['Games']

In [14]:
df19 = df19[['Teams','Games','Wins','Losses','Draws','WinningRatio','LosingRatio','DrawRatio']]

In [15]:
df_2019["home_win"] = df_2019["FTR"] == "H"

In [65]:
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [74]:
for n in df19.index :
    team = df19.loc[n,"Teams"]
    lossesbefore = "Games lost before by {}".format(team)
    winsbefore = "Games won before by {}".format(team)
    hwinsbefore = "Home games won before by {}".format(team)
    awinsbefore = "Away games won before by {}".format(team)
    hlossesbefore =  "Home games lost before by {}".format(team)
    alossesbefore =  "Away games lost before by {}".format(team)
    hgames = "Home games of {}".format(team)
    agames = "Away games of {}".format(team)
    homeadvantage = "Home advantage of {} before".format(team)
    overallfifa = "Avg overall FIFA20 players of {}".format(team)   
x = df_2019[[lossesbefore,winsbefore,hwinsbefore,awinsbefore,hlossesbefore,alossesbefore,hgames,agames,"Best odd is HomeTeam by +0.2",homeadvantage,overallfifa,"Home team has a better FIFA20 overall","Home team has more valuable players","Home team has more expensive players"]]
y = df_2019["home_win"]
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [75]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

C:\Users\yanis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [76]:
print(lr.coef_)
print(lr.intercept_)

[[-0.09544112  0.17569315  0.23806415 -0.062371   -0.04019455 -0.05524657
   0.01035161 -0.2034934   0.79400199 -0.6119734  -0.01380685  1.02665696
   0.11757544 -0.19139341]]
[-0.00020427]


In [77]:
y_pred = lr.predict(x_test)

In [78]:
confusion_matrix(y_test, y_pred)

array([[20,  8],
       [11, 11]], dtype=int64)

In [79]:
# Use score method to get accuracy of model
score = lr.score(x_test, y_test)
print(score)

0.62


In [19]:
for i in df_2019.index:
    if df_2019.loc[i,"home_win"] == True :
        df_2019.loc[i,"home_win"] = 1
    else :
        df_2019.loc[i,"home_win"] = 0

In [20]:
df_2019["away_win"] = df_2019["FTR"] == "A"

In [21]:
for i in df_2019.index:
    if df_2019.loc[i,"away_win"] == True :
        df_2019.loc[i,"away_win"] = 1
    else :
        df_2019.loc[i,"away_win"] = 0

In [22]:
for i  in df_2019.index :
    datetimecorr1 = datetime.datetime.strptime(df_2019.loc[i,'Date'], '%d/%m/%Y')
    df_2019.loc[i,"Datetime"] = datetimecorr1.date()

In [23]:
for i in df_2019.index:
    date = df_2019.loc[i,"Datetime"]
    temp_df = df_2019[df_2019["Datetime"] < date]
    df_2019.loc[i,"game_won_before_h"] = temp_df["home_win"].sum()

In [24]:
for i in df_2019.index :
    date = df_2019.loc[i,"Datetime"]
    temp_df = df_2019[df_2019["Datetime"] < date]
    df_2019.loc[i,"game_won_before_a"] = temp_df["away_win"].sum()

In [25]:
totalhomegoals = df_2019["FTHG"].sum()
totalawaygoals = df_2019["FTAG"].sum()
totalgoals = totalhomegoals + totalawaygoals
totalplayedgames = df19["Games"].sum() /2
avghomegoals = totalhomegoals / totalplayedgames
avgawaygoals = totalawaygoals / totalplayedgames
avggoals = totalgoals / totalplayedgames
print(avghomegoals,avgawaygoals,avggoals)

1.51 1.005 2.515


In [26]:
datelist = list(df_2019["Date"])

In [27]:
# Number of Home wins
for i in df19.index:
    team = df19.loc[i,"Teams"]
    winningratioh = df_2019.groupby("HomeTeam").mean().loc[team,"home_win"]
    df19.loc[i,"Homewins"] = winningratioh

In [28]:
# Number of Away wins
for i in df19.index:
    team = df19.loc[i,"Teams"]
    winningratioa = df_2019.groupby("AwayTeam").mean().loc[team,"away_win"]
    df19.loc[i,"Awaywins"] = winningratioa

In [29]:
# Number of Home Goals
for i in df19.index:
    team = df19.loc[i,"Teams"]
    homegoalsh = df_2019.groupby("HomeTeam").mean().loc[team,"FTHG"]
    df19.loc[i,"N° home goals per game"] = homegoalsh

In [30]:
# Number of Away Goals
for i in df19.index:
    team = df19.loc[i,"Teams"]
    awaygoalsa = df_2019.groupby("AwayTeam").mean().loc[team,"FTAG"]
    df19.loc[i,"N° away goals per game"] = awaygoalsa

In [31]:
# Number of Home Goals conceded
for i in df19.index:
    team = df19.loc[i,"Teams"]
    homegoalsa = df_2019.groupby("HomeTeam").mean().loc[team,"FTAG"]
    df19.loc[i,"N° home goals conceded per game"] = homegoalsa

In [32]:
# Number of Away Goals conceded
for i in df19.index:
    team = df19.loc[i,"Teams"]
    awaygoalsh = df_2019.groupby("AwayTeam").mean().loc[team,"FTHG"]
    df19.loc[i,"N° away goals conceded per game"] = awaygoalsh

In [33]:
# Number of Goals per game
for i in df19.index:
    team = df19.loc[i,"Teams"]
    homegoalsa = df_2019.groupby("HomeTeam").mean().loc[team,"FTHG"]
    awaygoalsa = df_2019.groupby("AwayTeam").mean().loc[team,"FTAG"]
    homegoalsa = df_2019.groupby("HomeTeam").mean().loc[team,"FTAG"]
    awaygoalsh = df_2019.groupby("AwayTeam").mean().loc[team,"FTHG"]
    df19.loc[i,"N° goals per game"] = (homegoalsa + awaygoalsa + homegoalsh + awaygoalsh)/2

In [34]:
df_2019["AvgH"].sum() / totalplayedgames

2.5741

In [35]:
df_2019["AvgA"].sum() / totalplayedgames

4.3328500000000005

In [36]:
df_2019["AvgD"].sum() / totalplayedgames

3.7491999999999996

In [37]:
df_2019["Avg>2.5"].sum() / totalplayedgames

2.08655

In [38]:
df_2019["Avg<2.5"].sum() / totalplayedgames

1.86375

In [39]:
df_2019

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,MaxCAHA,AvgCAHH,AvgCAHA,WinningTeam,LosingTeam,home_win,away_win,Datetime,game_won_before_h,game_won_before_a
0,F1,09/08/2019,19:45,Monaco,Lyon,0,3,A,0,2,...,2.01,2.01,1.87,Lyon,Monaco,0,1,2019-08-09,0.0,0.0
1,F1,10/08/2019,16:30,Marseille,Reims,0,2,A,0,0,...,1.97,1.98,1.89,Reims,Marseille,0,1,2019-08-10,0.0,1.0
2,F1,10/08/2019,19:00,Angers,Bordeaux,3,1,H,3,1,...,2.17,1.77,2.11,Angers,Bordeaux,1,0,2019-08-10,0.0,1.0
3,F1,10/08/2019,19:00,Brest,Toulouse,1,1,D,1,0,...,1.83,2.08,1.79,Draw,Draw,0,0,2019-08-10,0.0,1.0
4,F1,10/08/2019,19:00,Dijon,St Etienne,1,2,A,1,2,...,1.82,2.09,1.79,St Etienne,Dijon,0,1,2019-08-10,0.0,1.0
5,F1,10/08/2019,19:00,Montpellier,Rennes,0,1,A,0,1,...,2.02,1.93,1.94,Rennes,Montpellier,0,1,2019-08-10,0.0,1.0
6,F1,10/08/2019,19:00,Nice,Amiens,2,1,H,1,0,...,1.95,1.96,1.91,Nice,Amiens,1,0,2019-08-10,0.0,1.0
7,F1,11/08/2019,14:00,Lille,Nantes,2,1,H,1,0,...,2.06,1.96,1.91,Lille,Nantes,1,0,2019-08-11,2.0,4.0
8,F1,11/08/2019,16:00,Strasbourg,Metz,1,1,D,1,0,...,2.04,1.91,1.97,Draw,Draw,0,0,2019-08-11,2.0,4.0
9,F1,11/08/2019,20:00,Paris SG,Nimes,3,0,H,1,0,...,1.95,2.09,1.79,Paris SG,Nimes,1,0,2019-08-11,2.0,4.0


In [40]:
# Home game of each team 1-0
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name1 = "Home games of {}".format(team)
        if df_2019.loc[i,"HomeTeam"] == team :
            df_2019.loc[i,name1] = 1
        else :
            df_2019.loc[i,name1] = 0

In [41]:
# Away game of each team 1-0
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name1 = "Away games of {}".format(team)
        if df_2019.loc[i,"AwayTeam"] == team :
            df_2019.loc[i,name1] = 1
        else :
            df_2019.loc[i,name1] = 0

In [42]:
# Wins of each team 1-0
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name1 = "Wins of {}".format(team)
        if df_2019.loc[i,"WinningTeam"] == team :
            df_2019.loc[i,name1] = 1
        else :
            df_2019.loc[i,name1] = 0

In [43]:
# Wins before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        name1 = "Wins of {}".format(team)
        name2 = "Games won before by {}".format(team)
        df_2019.loc[i,name2] = temp_df[name1].sum()

In [44]:
# Home wins before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        temp_df2 = temp_df[temp_df["HomeTeam"] == team ]
        name1 = "Wins of {}".format(team)
        name2 = "Home games won before by {}".format(team)
        df_2019.loc[i,name2] = temp_df2[name1].sum()

In [45]:
# Away wins before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        temp_df2 = temp_df[temp_df["AwayTeam"] == team ]
        name1 = "Wins of {}".format(team)
        name2 = "Away games won before by {}".format(team)
        df_2019.loc[i,name2] = temp_df2[name1].sum()

In [46]:
# Losses of each team 1-0
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name1 = "Losses of {}".format(team)
        if df_2019.loc[i,"LosingTeam"] == team :
            df_2019.loc[i,name1] = 1
        else :
            df_2019.loc[i,name1] = 0

In [47]:
# Losses before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        name1 = "Losses of {}".format(team)
        name2 = "Games lost before by {}".format(team)
        df_2019.loc[i,name2] = temp_df[name1].sum()

In [48]:
# Home losses before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        temp_df2 = temp_df[temp_df["HomeTeam"] == team ]
        name1 = "Losses of {}".format(team)
        name2 = "Home games lost before by {}".format(team)
        df_2019.loc[i,name2] = temp_df2[name1].sum()

In [49]:
# Away losses before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        temp_df2 = temp_df[temp_df["AwayTeam"] == team ]
        name1 = "Losses of {}".format(team)
        name2 = "Away games lost before by {}".format(team)
        df_2019.loc[i,name2] = temp_df2[name1].sum()

In [50]:
# Home games played before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        name1 = "Home games of {}".format(team)
        name2 = "Home games of {} before".format(team)
        df_2019.loc[i,name2] = temp_df[name1].sum()

In [51]:
# Away games played before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        name1 = "Away games of {}".format(team)
        name2 = "Away games of {} before".format(team)
        df_2019.loc[i,name2] = temp_df[name1].sum()

In [52]:
# Avg odds before game 
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        temp_df2 = temp_df[temp_df["HomeTeam"] == team ]
        temp_df3 = temp_df[temp_df["AwayTeam"] == team ]
        name12 = "Away games of {} before".format(team)
        name11 = "Home games of {} before".format(team)
        name1 = "B365H"
        name4 = "B365A"
        name2 = "Avg home odd of {} in the past games".format(team)
        numbh= df_2019.loc[i, name11]
        df_2019.loc[i,name2] = temp_df2[name1].sum() / (numbh+1)
        numba= df_2019.loc[i, name12]
        name3 = "Avg away odd of {} in the past games".format(team)
        df_2019.loc[i,name3] = temp_df3[name4].sum() / (numba+1)

In [53]:
# Avg odds before game 2.0
for i in df_2019.index : 
    hometeam = df_2019.loc[i, "HomeTeam"]
    awayteam = df_2019.loc[i, "AwayTeam"]
    name1 = "Avg home odd of {} in the past games".format(hometeam)
    name2 = "Avg away odd of {} in the past games".format(awayteam)
    if df_2019.loc[i, name1] <= (df_2019.loc[i, name2]-0.2):
        df_2019.loc[i,"Best odd is HomeTeam by +0.2"] = 1
    else : 
        df_2019.loc[i,"Best odd is HomeTeam by +0.2"] = 0

In [54]:
# Home avantage before game 
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        temp_df2 = temp_df[temp_df["HomeTeam"] == team ]
        temp_df3 = temp_df[temp_df["AwayTeam"] == team ]   
        name12 = "Away games of {} before".format(team)
        name11 = "Home games of {} before".format(team)
        numbh= df_2019.loc[i, name11]
        numba= df_2019.loc[i, name12]
        goalsh = temp_df2["FTHG"].sum()
        goalsa = temp_df3["FTAG"].sum()
        concededh = temp_df2["FTAG"].sum()
        concededa = temp_df3["FTHG"].sum()
        name2 = "Home advantage of {} before".format(team)
        df_2019.loc[i,name2] = (goalsh-concededh)/(numbh+1)
        

In [55]:
# Avg overall FIFA20 player 
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name2 = "Avg overall FIFA20 players of {}".format(team)        
        temp_df = fifadata[fifadata["club"] == team]
        mean = temp_df["overall"].mean()
        df_2019.loc[i,name2] = mean

In [56]:
# HomeTeam better avg overall FIFA20 player 
for i in df_2019.index:
    hteam = df_2019.loc[i,"HomeTeam"]
    ateam = df_2019.loc[i,"AwayTeam"]
    name1 = "Avg overall FIFA20 players of {}".format(hteam) 
    name2 = "Avg overall FIFA20 players of {}".format(ateam) 
    hoverall = df_2019.loc[i,name1]
    aoverall = df_2019.loc[i,name2]
    name1 = "Home team has a better FIFA20 overall"
    if hoverall >= aoverall :
        df_2019.loc[i,name1] = 1
    else :
        df_2019.loc[i,name1] = 0

In [57]:
# Avg value of players
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name2 = "Avg value of players of {}".format(team)        
        temp_df = fifadata[fifadata["club"] == team]
        mean = temp_df["value_eur"].mean(skipna=True)
        df_2019.loc[i,name2] = mean
for i in df_2019.index:
    hteam = df_2019.loc[i,"HomeTeam"]
    ateam = df_2019.loc[i,"AwayTeam"]
    name1 = "Avg value of players of {}".format(hteam) 
    name2 = "Avg value of players of {}".format(ateam) 
    hoverall = df_2019.loc[i,name1]
    aoverall = df_2019.loc[i,name2]
    name1 = "Home team has more valuable players"
    if hoverall >= aoverall :
        df_2019.loc[i,name1] = 1
    else :
        df_2019.loc[i,name1] = 0

In [72]:
# Avg wage of players
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name2 = "Avg wage of players of {}".format(team)        
        temp_df = fifadata[fifadata["club"] == team]
        mean = temp_df["wage_eur"].mean(skipna=True)
        df_2019.loc[i,name2] = mean
for i in df_2019.index:
    hteam = df_2019.loc[i,"HomeTeam"]
    ateam = df_2019.loc[i,"AwayTeam"]
    name1 = "Avg wage of players of {}".format(hteam) 
    name2 = "Avg wage of players of {}".format(ateam) 
    hoverall = df_2019.loc[i,name1]
    aoverall = df_2019.loc[i,name2]
    name1 = "Home team has more expensive players"
    if hoverall >= aoverall :
        df_2019.loc[i,name1] = 1
    else :
        df_2019.loc[i,name1] = 0

In [73]:
df_2019

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,Avg wage of players of Angers,Avg wage of players of Nice,Avg wage of players of Toulouse,Avg wage of players of Amiens,Avg wage of players of Rennes,Avg wage of players of Brest,Avg wage of players of Dijon,Avg wage of players of Strasbourg,Avg wage of players of Metz,Home team has more expensive players
0,F1,09/08/2019,19:45,Monaco,Lyon,0,3,A,0,2,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,0.0
1,F1,10/08/2019,16:30,Marseille,Reims,0,2,A,0,0,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,1.0
2,F1,10/08/2019,19:00,Angers,Bordeaux,3,1,H,3,1,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,0.0
3,F1,10/08/2019,19:00,Brest,Toulouse,1,1,D,1,0,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,0.0
4,F1,10/08/2019,19:00,Dijon,St Etienne,1,2,A,1,2,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,0.0
5,F1,10/08/2019,19:00,Montpellier,Rennes,0,1,A,0,1,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,0.0
6,F1,10/08/2019,19:00,Nice,Amiens,2,1,H,1,0,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,1.0
7,F1,11/08/2019,14:00,Lille,Nantes,2,1,H,1,0,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,1.0
8,F1,11/08/2019,16:00,Strasbourg,Metz,1,1,D,1,0,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,1.0
9,F1,11/08/2019,20:00,Paris SG,Nimes,3,0,H,1,0,...,12290.322581,16034.482759,13206.896552,9562.5,24869.565217,8960.0,6166.666667,15642.857143,8296.296296,1.0


In [59]:
df19["Teamsc"] = df19["Teams"].apply(lambda x: x.replace(" ","_"))

In [60]:
for n in df19.index :
    team = df19.loc[n,"Teamsc"]
    nameteam = "df19_{}".format(team)
    for i in df_2019.index:
        df19_teamh = pd.DataFrame(df_2019.loc[df_2019.HomeTeam == team])
    for i in df_2019.index:
        df19_teama = pd.DataFrame(df_2019.loc[df_2019.AwayTeam == team])
    framesteam = [df19_teama, df19_teamh]
    result = pd.concat(framesteam)
    exec("%s = result.sort_index()" % (nameteam))

In [61]:
for n in df19.index :
    team = df19.loc[n,"Teamsc"]
    nameteam = "df19_{}".format(team)
    exec("%s['WINS']= %s['WinningTeam'] == team" %(nameteam,nameteam))

In [62]:
team = "Paris_SG"
nameteam = "df19_{}".format(team)
for i in df_2019.index:
    df19_teamh = pd.DataFrame(df_2019.loc[df_2019.HomeTeam == "Paris SG"])
for i in df_2019.index:
    df19_teama = pd.DataFrame(df_2019.loc[df_2019.AwayTeam == "Paris SG"])
framesteam = [df19_teama, df19_teamh]
result = pd.concat(framesteam)
exec("%s = result.sort_index()" % (nameteam))

In [63]:
team = "St_Etienne"
nameteam = "df19_{}".format(team)
for i in df_2019.index:
    df19_teamh = pd.DataFrame(df_2019.loc[df_2019.HomeTeam == "St Etienne"])
for i in df_2019.index:
    df19_teama = pd.DataFrame(df_2019.loc[df_2019.AwayTeam == "St Etienne"])
framesteam = [df19_teama, df19_teamh]
result = pd.concat(framesteam)
exec("%s = result.sort_index()" % (nameteam))

In [64]:
for n in df19.index :
    team = df19.loc[n,"Teamsc"]
    nameteam = "df19_{}".format(team)
    exec("%s.to_csv(r'C:\\Users\\yanis\\Downloads\\Foot test\\Clubs_2019\\%s_2019')" %(nameteam,nameteam))